In [7]:
import logging
import timm
import os
import random
import torch
import pandas as pd
import numpy as np
from albumentations.pytorch import ToTensorV2
import albumentations as A
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
import platform

os_name = platform.system()
if os_name == 'Windows':
    PRE_PATH = ''
elif os_name == 'Linux':
    PRE_PATH = '/kkh/'
elif os_name == 'Darwin': # 맥
    PRE_PATH = '/kkh/'
    
TRAIN_KR_IMAGE_PATH = PRE_PATH + 'data/train_kr'
TRAIN_KR_CSV_PATH = PRE_PATH + 'data/train_kr.csv'
MODEL_FILE = PRE_PATH + 'submission/0.9190/' + 'efficientnet_b4_Ep7_L_0.5274_A_0.9054_F1_0.9024.pt'
MODEL_SAVE_PATH = PRE_PATH + 'evaluation/'

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

NUM_CLASSES = 17
BATCH_SIZE = 32
NUM_WORKERS = 0  # 일단 0으로 설정하여 단일 프로세스로 수행

In [8]:
def create_directory_with_backup(path):
    try:
        if os.path.exists(path):
            backup_path = path + '_backup'
            os.rename(path, backup_path)
            print(f"Existing folder renamed to: {backup_path}")
        os.makedirs(path)
        print(f"Folder created: {path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Dataset class 정의
class ImageDataset(Dataset):
    def __init__(self, df, image_path, transform=None):
        self.df = df
        self.image_path = image_path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        try:
            name, target = self.df.iloc[idx]
            img = np.array(Image.open(os.path.join(self.image_path, name)).convert("RGB"))

            # 변환 적용
            if self.transform:
                img = self.transform(image=img)['image']

            return img, target

        except Exception as e:
            print(f"Error loading image {self.df.iloc[idx, 0]}: {e}")
            return torch.zeros(3, 380, 380), -1

# 이미지 변환 설정
def get_transforms():
    valid_transform = A.Compose([
        A.Resize(380, 380),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    return valid_transform

# 특정 클래스에 대한 confidence 계산 및 출력
def calculate_class_confidence(model, loader, target_class):
    model.eval()
    results = []
    with torch.no_grad():
        tbar = tqdm(loader, desc="Calculating Confidence")
        for batch_idx, (images, targets) in enumerate(tbar):
            images = images.to(device)
            outputs = model(images)
            confidences = torch.softmax(outputs, dim=1)
            target_confidences = confidences[:, target_class].detach().cpu().numpy()
            
            # 이미지 인덱스를 사용하여 정확한 파일 이름을 추적
            for idx, confidence in enumerate(target_confidences):
                # 원래 데이터프레임의 인덱스를 계산
                df_idx = batch_idx * loader.batch_size + idx
                filename = loader.dataset.df.iloc[df_idx, 0]
                results.append((filename, confidence))
    
    results = sorted(results, key=lambda x: x[1], reverse=True)
    for filename, confidence in results:
        if confidence < 0.8:
            print(f"Image: {filename}, Confidence: {confidence:.4f}")


# 메인 함수
def run_confidence_check(target_class):
    train_df = pd.read_csv(PRE_PATH + 'data/train_kr_aug_8_patch2.csv')

    # 클래스가 3인 이미지들만 필터링
    target_df = train_df[train_df['target'] == target_class]

    valid_transform = get_transforms()

    train_dataset = ImageDataset(target_df, PRE_PATH + 'data/train_kr_aug_8_patch2/', transform=valid_transform)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

    model_name = 'tf_efficientnet_b4'
    model = timm.create_model(model_name, pretrained=False, num_classes=NUM_CLASSES, in_chans=3)

    # map_location 인자를 사용하여 모델을 CPU에서 로드
    model.load_state_dict(torch.load(MODEL_FILE, map_location=device))
    model = model.to(device)

    # 특정 클래스에 대한 confidence 계산 및 출력
    calculate_class_confidence(model, train_loader, target_class)

In [9]:
# target_class = 3  # 여기서 특정 클래스를 설정
for target_class in range(17):
    run_confidence_check(target_class)

/tmp/ipykernel_468119/2030146830.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_FILE, map_location=device))
Calculating Confiden

Image: 3ffe6ae0f539496d_계좌번호.jpg, Confidence: 0.7880
Image: 0fa6bb7b61883601_계좌번호.jpg, Confidence: 0.7823
Image: 0cdf7f30dbeacb1c_계좌번호.jpg, Confidence: 0.7775
Image: 108c622ecd7a9ea0_계좌번호.jpg, Confidence: 0.7721
Image: 39e47698e2ffe3d6_계좌번호.jpg, Confidence: 0.7655
Image: 07a0a849cab135e0_계좌번호.jpg, Confidence: 0.7414
Image: 5cba776cef24af8a_계좌번호.jpg, Confidence: 0.7365
Image: 07a0a849cab135e0_계좌번호.jpg, Confidence: 0.7241
Image: 300c5be1070fd0ca_계좌번호.jpg, Confidence: 0.7173
Image: 135c81e626dbfde1_계좌번호.jpg, Confidence: 0.7171
Image: 3e7fdfd21ce2dd75_계좌번호.jpg, Confidence: 0.7110
Image: 1cc6427133ca0bc8_계좌번호.jpg, Confidence: 0.6876
Image: 30b2df34210bef90_계좌번호.jpg, Confidence: 0.6810
Image: 097bf6816bf11df3_계좌번호.jpg, Confidence: 0.6758
Image: 39132ee7af16925f_계좌번호.jpg, Confidence: 0.6734
Image: 2916e5db8a60d01a_계좌번호.jpg, Confidence: 0.6703
Image: 2b79f6f8cce0368c_계좌번호.jpg, Confidence: 0.6644
Image: 11f522c4a00d4c8c_계좌번호.jpg, Confidence: 0.6622
Image: 2d1381ebe8a01fa2_계좌번호.jpg, Confidence: 

/tmp/ipykernel_468119/2030146830.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_FILE, map_location=device))
Calculating Confiden

Image: 5e6b87eefb3f06a3_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.7977
Image: 9df479a4a43abb1c_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.7968
Image: 0a4adccbb7fe73e0_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.7864
Image: a1deb8f4332c8823_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.7856
Image: 0a4adccbb7fe73e0_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.7839
Image: 3d37e6b01e0cde9d_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.7815
Image: aaff35477765a02f_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.7811
Image: 3d37e6b01e0cde9d_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.7323
Image: 2ca6dd9543332aea_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.6941
Image: 7f23b9ae37e539ea_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.6743
Image: a683764ced480eb3_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.6738
Image: a683764ced480eb3_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.6092
Image: 485508cdbc358c78_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.5935
Image: 9a36a151cf95bb29_건강보험 임신출산 진료비 지급 신청서.jpg, Confidence: 0.5565
Image: 1857d71e467f27a2_건강보험 임신출산 

/tmp/ipykernel_468119/2030146830.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_FILE, map_location=device))
Calculating Confiden

...

/tmp/ipykernel_468119/2030146830.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_FILE, map_location=device))
Calculating Confiden

Image: 0764b11d22a5bf2b_이력서.jpg, Confidence: 0.7993
Image: 3c99358b35b7e8fa_이력서.jpg, Confidence: 0.7983
Image: 3e75ed82d7e027a2_이력서.jpg, Confidence: 0.7967
Image: 0764b11d22a5bf2b_이력서.jpg, Confidence: 0.7951
Image: 46f1b9d6cb5864f5_이력서.jpg, Confidence: 0.7930
Image: 0e73f69b538fb276_이력서.jpg, Confidence: 0.7880
Image: 0f1936616378ab3a_이력서.jpg, Confidence: 0.7869
Image: 4e63bb0e147078b6_이력서.jpg, Confidence: 0.7862
Image: 493382458cfa36b8_이력서.jpg, Confidence: 0.7769
Image: 0787007d9064369b_이력서.jpg, Confidence: 0.7758
Image: 4e63bb0e147078b6_이력서.jpg, Confidence: 0.7748
Image: 0764b11d22a5bf2b_이력서.jpg, Confidence: 0.7702
Image: 11d2eeafa3170c62_이력서.jpg, Confidence: 0.7656
Image: 12f0216cfc02be14_이력서.jpg, Confidence: 0.7612
Image: 4b90856f199fba68_이력서.jpg, Confidence: 0.7609
Image: 0f1936616378ab3a_이력서.jpg, Confidence: 0.7595
Image: 0e73f69b538fb276_이력서.jpg, Confidence: 0.7505
Image: 0764b11d22a5bf2b_이력서.jpg, Confidence: 0.7493
Image: 024fe478044874ab_이력서.jpg, Confidence: 0.7424
Image: 0764b

/tmp/ipykernel_468119/2030146830.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_FILE, map_location=device))
Calculating Confiden

Image: 840c53f776730ffe_소견서.jpg, Confidence: 0.7983
Image: 2e3550bead1a72cb_소견서.jpg, Confidence: 0.7963
Image: 76ff32bef566bd38_소견서.jpg, Confidence: 0.7962
Image: 78b3f4f8fde4d8e5_소견서.jpg, Confidence: 0.7928
Image: 0f3ac436363ae727_소견서.jpg, Confidence: 0.7895
Image: 91f89a9a3238fc94_소견서.jpg, Confidence: 0.7890
Image: 3696b2ffaa6147e6_소견서.jpg, Confidence: 0.7881
Image: 68465df7e54034e7_소견서.jpg, Confidence: 0.7863
Image: 22819e513decca0f_소견서.jpg, Confidence: 0.7855
Image: 93964b61ead8e03e_소견서.jpg, Confidence: 0.7847
Image: 012913977fd1d980_소견서.jpg, Confidence: 0.7837
Image: 706f88bd829080ed_소견서.jpg, Confidence: 0.7826
Image: 8db8f37ec84b4f8f_소견서.jpg, Confidence: 0.7824
Image: 2edee3a4605a6992_소견서.jpg, Confidence: 0.7822
Image: 5ba19592cee8212c_소견서.jpg, Confidence: 0.7819
Image: 37f9414beea68229_소견서.jpg, Confidence: 0.7816
Image: 2bbdb0a519013af2_소견서.jpg, Confidence: 0.7815
Image: 0f3ac436363ae727_소견서.jpg, Confidence: 0.7795
Image: 4bdec47234b71ffb_소견서.jpg, Confidence: 0.7762
Image: 5ba19

/tmp/ipykernel_468119/2030146830.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_FILE, map_location=device))
Calculating Confiden